In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import imodelsx.metrics
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import numpy as np

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
checkpoint = 'gpt2'
m = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=100, replace=False))

In [ ]:
from torch.autograd import grad
import torch
import torch
from torch.autograd.functional import jacobian
from torch.func import jacfwd

# Load pre-trained GPT-2 model
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()


def forward_embs(embs):
    output = model(inputs_embeds=embs)
    return output.logits


# Input text example
input_text = "Example"

# Tokenize and obtain model output
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids = tokenizer.encode(input_text, return_tensors="pt")
input_embs = model.transformer.wte(input_ids)
# output = model(inputs_embeds=input_embs)

In [ ]:
input_embs = input_embs.to('cuda')
model = model.to('cuda')

In [ ]:
def forward_embs(embs):
    # torch.zeros(1, 6, 768).shape
    output = model(inputs_embeds=embs)
    return output.logits

In [ ]:
# jac = jacobian(
# forward_embs, input_embs, strategy='forward-mode', vectorize=True)
jac = jacfwd(forward_embs)(input_embs)
jac = jac.squeeze()

In [ ]:
jac.shape

In [ ]:
jac_input = jac @ model.transformer.wte.weight.T  # output x input
jac_input.device

In [ ]:
jac_input = jac_input.cpu().detach().numpy()

In [32]:
jac_input[:3, :3]

array([[-1.8717968 ,  0.37471896, -1.5080535 ],
       [-2.5702996 ,  0.63906926, -1.5047908 ],
       [-2.5230765 ,  0.21229628, -0.16100714]], dtype=float32)

In [ ]:
plt.imshow(jac_input)

# Take multi-dimensional grad

In [ ]:
# Define the input tensor
x = torch.tensor(torch.normal(0, 1, size=(1, 5)), requires_grad=True)

# Compute the output of the function
param = torch.ones(5, 4)
param[0, :] = 2


def f(x):
    return x @ param


# Compute the gradient of y with respect to x
# gradient = torch.autograd.grad(y, x, torch.ones_like(y))[0]
# gradient = jacobian(f, x)[0]
gradient = jacfwd(f)(x)

print("Input x:", x)
print("Output y:", f(x))
print("Jacob dy/dx:", gradient)